In [23]:
from gurobipy import GRB
import gurobipy as gb
import pandas as pd
import numpy as np

In [24]:
# Create the optimization model
model = gb.Model("Question 3: Transportation")

In [25]:
cost = [0, 20, 0, 0, 25, 0, 0, 
        0, 0, 0, 0, 30, 0, 45,
        20, 0, 0, 0, 0, 35, 0,
        0, 30, 0, 0, 0, 0, 0,
        0, 0, 25, 15, 0, 28, 0,
        0, 0, 0, 0, 0, 0, 12,
        0, 0, 0, 27, 0, 0, 0]

In [26]:
cost = np.array(cost).reshape(7, -1)

In [27]:
cost

array([[ 0, 20,  0,  0, 25,  0,  0],
       [ 0,  0,  0,  0, 30,  0, 45],
       [20,  0,  0,  0,  0, 35,  0],
       [ 0, 30,  0,  0,  0,  0,  0],
       [ 0,  0, 25, 15,  0, 28,  0],
       [ 0,  0,  0,  0,  0,  0, 12],
       [ 0,  0,  0, 27,  0,  0,  0]])

In [28]:
# From = {7 sites} and To = {7 sites}.
x = model.addVars(7, 7, lb=0, vtype=GRB.CONTINUOUS, name="Transportation Between Hubs")

In [29]:
objective = gb.quicksum(cost[i,j] * x[i,j] for i in range(7) for j in range (7))
model.setObjective(objective, GRB.MINIMIZE)

In [30]:
model.addConstr(x[0,1] + x[0,4] - x[2,0] == -0.08 * 1400, name="Hub 1 constraint")
model.addConstr(x[1,4] + x[1,6] - x[0,1] - x[3,1] == 0.05 * 1400, name="Hub 2 constraint")
model.addConstr(x[2,0] + x[2,5] - x[4,2] == 0.03 * 1400, name="Hub 3 constraint")
model.addConstr(x[3,1] - x[4,3] - x[6,3] == -0.03 * 1400, name="Hub 4 constraint")
model.addConstr(x[4,2] + x[4,3] + x[4,5] - x[0,4] - x[1,4] == 0.02 * 1400, name="Hub 5 constraint")
model.addConstr(x[5,6] - x[2,5] - x[4,5] == -0.05 * 1400, name="Hub 6 constraint")
model.addConstr(x[6,3] - x[1,6] - x[5,6] == 0.06 * 1400, name="Hub 7 constraint")

<gurobi.Constr *Awaiting Model Update*>

In [31]:
model.addConstr(x[1,4] + x[1,6] + x[2,1] + x[2,5] <= 2*(x[3,1] + x[4,2] + x[4,3] + x[4,5]), name="First Logical Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [32]:
model.addConstr(gb.quicksum(x[i,j] for i in range (5) for j in range (7)) >= 0.05 * 1400, name="Second Logical Constraint Part 1")
model.addConstr(gb.quicksum(x[i,j] for i in range (5) for j in range (7)) <= 0.50 * 1400, name="Second Logical Constraint Part 2")

<gurobi.Constr *Awaiting Model Update*>

In [33]:
# Optimally solve the problem
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 10 rows, 49 columns and 102 nonzeros
Model fingerprint: 0x4ffa3e70
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 7e+02]
Presolve removed 8 rows and 46 columns
Presolve time: 0.00s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0878000e+04   8.750000e+00   0.000000e+00      0s
       1    1.2838000e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.283800000e+04


In [34]:
# Print the decision variables
print(model.printAttr('X'))


    Variable            X 
-------------------------
Transportation Between Hubs[0,0]          294 
Transportation Between Hubs[1,4]          112 
Transportation Between Hubs[2,0]          112 
Transportation Between Hubs[3,1]           42 
Transportation Between Hubs[4,2]           70 
Transportation Between Hubs[4,5]           70 
Transportation Between Hubs[6,3]           84 
None


In [35]:
# Value of the objective function
print("Total Transportation cost: ", round(model.objVal, 2))

Total Transportation cost:  12838.0
